In [2]:
import sys, time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import kornia
import math
from custom_models import *
from custom_datasets import *
from custom_transforms import *
from utils import *
from torch.utils.data import SubsetRandomSampler, Sampler, Subset, ConcatDataset, random_split

/home/rayeh/miniconda3/envs/autodo/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
paramM = nn.Parameter(torch.ones(3,3), requires_grad=True)
paramM = paramM.cuda()
a = torch.rand(1).cuda()

In [3]:
img_scale = 1.
dataset_0to4 = MedDataset('/mnt/Nami/Med_patch/8192_512_0to4/Good', '/mnt/Nami/Med_patch/8192_512_0to4/Good_mask', img_scale)
total_images = len(dataset_0to4)
val_images_1 = 25
train_images = total_images - val_images_1
train_dataset, val_dataset_1 = random_split(dataset_0to4, [train_images,val_images_1])
#test data: WSI 6-10
dataset_5to9 = MedDataset('/mnt/Nami/Med_patch/8192_512_5to9/Good', '/mnt/Nami/Med_patch/8192_512_5to9/Good_mask', img_scale)
_, val_dataset_2 = random_split(dataset_5to9, [len(dataset_5to9)-val_images_1,val_images_1])
val_dataset = val_dataset_1+val_dataset_2
test_dataset = MedDataset('/mnt/Nami/Med_patch/8192_512_10/Good', '/mnt/Nami/Med_patch/8192_512_10/Good_mask', img_scale)
valid_images = len(val_dataset)

In [2]:
K = 7
N = 5
B = 2
magn = 5
grad = True
#預設機率與強度超參數的初始值
magnNorm = torch.ones(1)*magn/10.0 # normalize to 10 like in RandAugment
probNorm = torch.ones(1)*1/(K-2) # 1/(K-2) probability
magnLogit = torch.log(magnNorm/(1-magnNorm)) # convert to logit
probLogit = torch.log(probNorm/(1-probNorm)) # convert to logit
# affine transforms (mid, range)
angle = [0.0, 30.0] # [-30.0:30.0] rotation angle
trans = [0.0, 0.45] # [-0.45:0.45] X/Y translate
shear = [0.0, 0.30] # [-0.30:0.30] X/Y shear
scale = [1.0, 0.50] # [ 0.50:1.50] X/Y scale
# color transforms (mid, range)
bri = [0.0, 0.9] # [-0.9:0.9] brightness
con = [1.0, 0.9] # [0.1:1.9] contrast
sat = [0.1, 1.9] # [-0.30:0.30] saturation
#hue = [1.0, 0.50] # [ 0.70:1.30] hue
#gam = [1.0, 0.50] # [ 0.70:1.30] gamma
# actP: 機率的激活函數，actM: 強度的激活函數
actP = nn.Sigmoid()
actM = nn.Sigmoid()
# 建立機率與強度的可學習超參數矩陣 shape = (擴增方法數量*資料數量)
paramP = nn.Parameter(probLogit*torch.ones(K,N), requires_grad=grad)
paramM = nn.Parameter(magnLogit*torch.ones(K,N), requires_grad=grad)

In [3]:
idx = [0,3]
paramPos = torch.log(    actP(paramP[:,idx])) # [-Inf:0] [擴增方法數, data point number] 取針對該資料的使用機率之參數
paramNeg = torch.log(1.0-actP(paramP[:,idx])) # [-Inf:0] [擴增方法數, data point number] 取針對該資料的不使用機率之參數
paramM2 = actM(paramM[:,idx]) # (K+J)xB [0:1], default=magn

In [4]:
paramP = torch.cat([paramPos.view(-1,1), paramNeg.view(-1,1)], dim=1) # B*(K+J)x2
# reparametrize probabilities and magnitudes
sampleP = F.gumbel_softmax(paramP, tau=1.0, hard=True) # B*(K+J)x2
sampleP = sampleP[:,0]
sampleP = sampleP.reshape(K,B)
# # reparametrize magnitudes
# sampleM = paramM[:K] * torch.rand(K,B) # KxB, prior: U[0,1]
G = torch.randn(K,B)
sampleM = paramM2 * G # KxB, prior: N(0,1)

In [5]:
print(G)

tensor([[-0.7640,  0.4978],
        [ 0.1357,  1.2249],
        [ 0.1424, -0.5961],
        [-1.2880, -0.6164],
        [ 0.1763,  0.3200],
        [ 1.7368, -0.5983],
        [ 0.9187,  3.0334],
        [ 1.0921, -1.4722],
        [ 0.3853, -0.1416]])


In [6]:
print(sampleM)

tensor([[-0.3820,  0.2489],
        [ 0.0678,  0.6124],
        [ 0.0712, -0.2981],
        [-0.6440, -0.3082],
        [ 0.0881,  0.1600],
        [ 0.8684, -0.2991],
        [ 0.4593,  1.5167],
        [ 0.5461, -0.7361],
        [ 0.1927, -0.0708]], grad_fn=<MulBackward0>)
